# Variables

In [437]:
OPENAI_API_KEY='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'
FIRECRAWL_API_KEY='fc-e36282d62d1c4ee8971ec0649f3862c7'
BASE_URL='https://api.groq.com/openai/v1'
MODEL="meta-llama/llama-4-scout-17b-16e-instruct"

# Import Modules

In [438]:
import sys
sys.path.append('..')
print(sys.path)
from tools.fetching_description_from_huggingface import fetching_description_from_huggingface
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
from langchain_core.output_parsers import PydanticOutputParser
import pandas as pd
import pprint

['/usr/local/lib/python312.zip', '/usr/local/lib/python3.12', '/usr/local/lib/python3.12/lib-dynload', '', '/home/hieuclc/ise_chenla/venv/lib/python3.12/site-packages', '..', '..', '..', '..', '..', '..', '..']


# Testing

In [439]:
df_task = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

In [440]:
df_task

,STT,Task,Validation data
0,1,"""Bối cảnh của vấn đề:\nNhận dạng chữ viết tay ...",Link
1,2,Bối cảnh của vấn đề:\nTrong thời đại thông tin...,Link
2,3,Bối cảnh của vấn đề:\nĐây là một nhiệm vụ hồi ...,Link
3,4,Bối cảnh của vấn đề:\nĐiều quan trọng là các c...,Link
4,5,Bối cảnh của vấn đề:\nTrong nhiều ứng dụng thự...,Link


In [441]:
task = df_task["Task"][4]

In [442]:
task

'Bối cảnh của vấn đề:\nTrong nhiều ứng dụng thực tế như tổng hợp nội dung hình ảnh, phân loại album, hay trích xuất đặc trưng nội dung cho ảnh tổ hợp, cần xác định chủ đề chính của một ảnh ghép từ nhiều ảnh con. Bài toán này đòi hỏi ứng dụng không chỉ nhận diện được nội dung của từng ảnh con, mà còn cần xác định chủ đề chiếm ưu thế trong toàn bộ ảnh tổng thể.\n\nYêu cầu cụ thể cần đạt được:\nMỗi đầu vào là một ảnh tổ hợp, ghép từ 9 ảnh màu nhỏ (3x3).\nNhiệm vụ của ứng dụng là:\nXác định chủ đề chiếm số lượng nhiều nhất trong 9 ảnh con.\nTrường hợp có nhiều chủ đề xuất hiện với tần suất bằng nhau, có thể chọn một chủ đề bất kỳ trong số đó làm kết quả.\n\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\nMột thư mục có tên là "images" chứa các ảnh màu RGB (288x288), được ghép từ 9 ảnh RGB con (32x32) (3 hàng × 3 cột).\n\nĐịnh dạng kết quả đầu ra mong muốn cho bài toán tổng thể:\nFile output.csv mỗi hàng là kết quả dự đoán mỗi ảnh\ncó các cột:\nfile_name: tên file ảnh\nprediction: nhãn củ

In [443]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


## Tools

In [444]:
def web_scraper(url: str) -> str:
    """Get more details about the model using the model url"""
    # scraper = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
    # scrape_status = scraper.scrape_url(
    #     url,
    #     formats=['markdown']
    # )
    # return scrape_status.markdown
    result = fetching_description_from_huggingface(url)
    return result

In [445]:
class WebScraperParameters(BaseModel):
    url: str

In [446]:
res = web_scraper('https://huggingface.co/thanhtlx/image_classification_01')

In [447]:
tools = [
    Tool(
        name="web_scrapper",
        func=web_scraper,
        description="Use to get more detail about the model",
        args_schema=WebScraperParameters
    )
]

## Model

In [448]:
llm = ChatOpenAI( 
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0.3
)

In [449]:
class Tasks(BaseModel):
    subtask_one: str = Field(
        ...,
        description="Detailed description of task 1"
    )
    subtask_two: str = Field(
        ...,
        description="Detailed description of task 2"
    )
    subtask_three: str = Field(
        ...,
        description="Detailed description of task 3"
    )
    subtask_four: str = Field(
        ...,
        description="Detailed description of task 4"
    )

In [450]:
class AgentState(TypedDict):
    task: HumanMessage | None
    data: str | None
    messages: Annotated[BaseMessage, add_messages]
    subtasks: Tasks

In [451]:
def classify_agent(state: AgentState) -> AgentState:
    structured_llm = llm.with_structured_output(Tasks)
    system_prompt = """
        You are an expert in task formulation. Given a machine learning task, you will
        help divide the task into these subtasks:
        - Subtask1: for identifying task description and classify the problem type.
        - Subtask2: for identifying the problem requirements as described in the "Yêu cầu cụ thể cần đạt được" (or other equivalent) section, including: problem requirements, expected input and expected output.
        - Subtask3: for identifying the input format of the problem, as being described in the "Định dạng dữ liệu đầu vào cho bài toán tổng thể" (or other equivalent) section.
        - Subtask4: for identifying the output format of the problem with the given classes, as being described in the "Định dạng kết quả đầu ra mong muốn cho bài toán tổng thể" (or other equivalent) section.

        You **MUST** follow these guidelines:
        - Do not mistake the output classes with the one described in the problem requirements. The output classes **MUST** contain the classes described in the "Định dạng kết quả đầu ra mong muốn cho bài toán tổng thể" section.
        - Describe the subtasks as specific as possible.
        """
    messages = [SystemMessage(content=system_prompt)]
    messages.append(state['task'])
    result = structured_llm.invoke(messages)
    state['subtasks'] = result
    return state

In [452]:
task = df_task["Task"][2]

In [453]:
state = {
    "task": HumanMessage(content=task),
}

In [454]:
state = classify_agent(state)

In [455]:
pprint.pprint(state['subtasks'].subtask_one)

('Xác định mô tả bài toán và phân loại loại bài toán. Bài toán này là một '
 'nhiệm vụ hồi quy, nhưng mục tiêu cuối cùng là phân loại giá trị trung bình '
 "của nhà ở vào các nhóm 'rẻ', 'trung bình', hay 'đắt'.")


In [456]:
pprint.pprint(state['subtasks'].subtask_two)

('Xác định yêu cầu cụ thể của bài toán. Đầu vào là một list các features của '
 'khu vực, bao gồm MedInc, HouseAge, AveRooms, AveBedrms, Population, '
 'AveOccup, Latitude, Longitude. Đầu ra là nhãn của nhà ở khu vực đó thuộc về '
 "một trong ba nhóm: 'low' (rẻ), 'medium' (trung bình), hay 'high' (đắt).")


In [457]:
pprint.pprint(state['subtasks'].subtask_three)

('Xác định định dạng dữ liệu đầu vào. Dữ liệu đầu vào là một file test.csv, '
 'mỗi hàng đại diện cho các features của một khu vực, bao gồm các cột: MedInc, '
 'HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude, '
 'ID.')


In [458]:
pprint.pprint(state['subtasks'].subtask_four)

('Xác định định dạng kết quả đầu ra. Kết quả đầu ra là một file output.csv, '
 'mỗi hàng đại diện cho kết quả dự đoán của một khu vực, bao gồm các cột: ID, '
 "MedHouseVal với các nhãn 'low', 'medium', hay 'high'.")


# Modeling Agent

In [459]:
class ModelInfo(BaseModel):
    model: str = Field(..., description="Model name")
    url: str = Field(..., description="URL to the model")

class ModelSelection(BaseModel):
    models: List[ModelInfo] = Field(..., description="List of available models with name and URL")

In [460]:
model_selection_parser = PydanticOutputParser(pydantic_object=ModelSelection)

In [461]:
def model_selection_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the suitable model for a given task.
                    Given:
                    - A user description about the task
                    - A list of available model (names and links)
                    Your job is to choose ALL suitable models provided in the list for user's specific tasks.
                    Return the answer in the format
                    {structured_output}
                    Here are the list of model:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "{input}")
    ]).partial(structured_output=model_selection_parser.get_format_instructions())

    chain = prompt | llm | model_selection_parser

    result = chain.invoke({"input": task, "model_list": format_model_list(df_model)})
    return result

In [462]:
res = model_selection_agent(state)

/tmp/ipykernel_14809/7332665.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  desc = str(row[1]).replace("{", "{{").replace("}", "}}")
/tmp/ipykernel_14809/7332665.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = str(row[2])


In [463]:
pprint.pprint(res.models)

[ModelInfo(model='Dự đoán giá trung bình nhà ở các quận Cali', url='https://huggingface.co/zhaospei/Model_2')]


In [464]:
def model_selection_using_tool_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the **Best-fit model** for a given task.
                    Given:
                    - A user task description
                    - A list of available models (names and url)
                    - A tool that fetches model details from a provided URL (it help gathers model description, input/output format, code sample usage)

                    Your job is to:
                    1. Use the **provided tool** to retrieve real details about each candidate model:
                        - ✅ Model description
                        - 📥 Input format
                        - 📤 Output format
                        - 🛠️ Library requirements
                        - 🧪 Code sample (usage code snippet)
                    2. Select the best model for user given task based on the data that the tool gives you.

                    ⚠️ VERY IMPORTANT RULES

                    - ❌ DO NOT assume or invent any part of the model's description, input/output format, or code
                    - ❌ DO NOT generate fake code or use your own knowledge about the model
                    - ✅ ONLY use the **actual output** returned from the tool
                    - ✅ Include tool content in your final answer exactly as returned (especially code)

                    ## 🧠 Output Structure (Final Answer)

                    After retrieving tool results, choose ONE best-fit model and output the result in the following format:

                    **✅ Model name and link**  
                    `<model name>` — `<link>`

                    **📝 Description (from tool):**  
                    <model description>

                    **📥 Input format (from tool):**  
                    <description of expected input>

                    **📤 Output format (from tool):**  
                    <description of model output>

                    **🛠️ Library Requirements (from tool) **
                    <requirements to use the model>

                    **🧪 Example code (from tool):**  
                    ```python
                    <exact code snippet from tool>

                    Here are the list of models and there corresponding URL:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("ai", "{agent_scratchpad}"),
        ("human", "{input}")
    ])

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    task_input = state["task"].content if isinstance(state["task"], SystemMessage) else state["task"]

    # result = executor.invoke({"input": task_input})
    result = executor.invoke({"input": task_input, "model_list": res.models})

    return {"output": result["output"]}

In [465]:
res2 = model_selection_using_tool_agent(state)



> Entering new AgentExecutor chain...

Invoking: `web_scrapper` with `{'url': 'https://huggingface.co/zhaospei/Model_2'}`



🏠 Mô hình Wide & Deep Neural Network - Dự đoán Giá Nhà California
📝 Mô tả
Đây là một mô hình Wide & Deep Neural Network được huấn luyện trên tập dữ liệu California Housing để dự đoán giá nhà trung bình (MedHouseVal). Mô hình được xây dựng bằng PyTorch, dựa trên kiến trúc trong cuốn Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow của Aurélien Géron.
📌 Nhiệm vụ
Dự đoán giá nhà dựa trên dữ liệu bảng (tabular regression) với 8 đặc trưng đầu vào.
📥 Đầu vào
Số chiều: [batch_size, 8] 
Kiểu dữ liệu: torch.FloatTensor 
Các đặc trưng đầu vào:
'MedInc' – Thu nhập trung vị
'HouseAge' – Tuổi trung bình của căn nhà
'AveRooms' – Số phòng trung bình
'AveBedrms' – Số phòng ngủ trung bình
'Population' – Dân số
'AveOccup' – Số người trung bình trên mỗi hộ
'Latitude' – Vĩ độ
'Longitude' – Kinh độ
📤 Đầu ra
Kiểu: torch.FloatTensor có shape [batch_size, 1] 
Ý nghĩa

In [466]:
pprint.pprint(res2['output'])

('**✅ Model name and link**  \n'
 ' `Dự đoán giá trung bình nhà ở các quận Cali` — '
 '`https://huggingface.co/zhaospei/Model_2`\n'
 '\n'
 ' **📝 Description (from tool):**  \n'
 ' Đây là một mô hình Wide & Deep Neural Network được huấn luyện trên tập dữ '
 'liệu California Housing để dự đoán giá nhà trung bình (MedHouseVal). Mô hình '
 'được xây dựng bằng PyTorch, dựa trên kiến trúc trong cuốn Hands-On Machine '
 'Learning with Scikit-Learn, Keras, and TensorFlow của Aurélien Géron.\n'
 '\n'
 ' **📥 Input format (from tool):**  \n'
 ' Số chiều: [batch_size,8] \n'
 ' Kiểu dữ liệu: torch.FloatTensor \n'
 ' Các đặc trưng đầu vào:\n'
 " - 'MedInc' – Thu nhập trung vị\n"
 " - 'HouseAge' – Tuổi trung bình của căn nhà\n"
 " - 'AveRooms' – Số phòng trung bình\n"
 " - 'AveBedrms' – Số phòng ngủ trung bình\n"
 " - 'Population' – Dân số\n"
 " - 'AveOccup' – Số người trung bình trên mỗi hộ\n"
 " - 'Latitude' – Vĩ độ\n"
 " - 'Longitude' – Kinh độ\n"
 '\n'
 ' **📤 Output format (from tool):**  \n'
 ' 

## Output Parser Agent

In [467]:
class ModelingOutput(BaseModel):
    model_description: str = Field(
        ...,
        description="The detailed description of the model."
    )
    model_input_format: str = Field(
        ...,
        description="Model's detailed input format"
    )
    model_output_format: str = Field(
        ...,
        description="Model's detailed output format"
    )
    model_requirements: str = Field(
        ...,
        description="Requirement libraries need to install to use the model."
    )
    model_sample_code: str = Field(
        ...,
        description="Example code to use the model"
    )

In [468]:
parser = PydanticOutputParser(pydantic_object=ModelingOutput)

In [469]:
output_parser_llm = ChatOpenAI(
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0
)

In [470]:
def output_parser_agent(state: AgentState):
    system_prompt = """
                    You are a smart AI tasked with extracting structured technical details about a machine learning model from a reasoning result.
                    You are given a detailed text description about a model.
                    Your goal is to fill the following fields based on the text:

                    - `model_description`: A detailed explanation of what the model is and what it does.
                    - `model_input_format`: A detailed description of the model's input format, including dimensions, data types, and expected preprocessing if mentioned.
                    - `model_output_format`: A detailed description of the output format including dimensions, data types, label name and its meaning.
                    - `model_requirements`: A detailed description about the requirements needed to be sastified in order to use the model
                    - `model_sample_code`: A sample code on how to use the model.

                    You MUSTN'T create any data on your own, only using the data provided in the text.

                    Return the data as a JSON object matching the following structure:
                    {formatted_output}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        ('human', "{input}")
    ]).partial(formatted_output=parser.get_format_instructions())
    chain = prompt | llm | parser
    response = chain.invoke({"input": res2['output']})
    return response

In [471]:
ans = output_parser_agent(state)

In [472]:
pprint.pprint(res2['output'])

('**✅ Model name and link**  \n'
 ' `Dự đoán giá trung bình nhà ở các quận Cali` — '
 '`https://huggingface.co/zhaospei/Model_2`\n'
 '\n'
 ' **📝 Description (from tool):**  \n'
 ' Đây là một mô hình Wide & Deep Neural Network được huấn luyện trên tập dữ '
 'liệu California Housing để dự đoán giá nhà trung bình (MedHouseVal). Mô hình '
 'được xây dựng bằng PyTorch, dựa trên kiến trúc trong cuốn Hands-On Machine '
 'Learning with Scikit-Learn, Keras, and TensorFlow của Aurélien Géron.\n'
 '\n'
 ' **📥 Input format (from tool):**  \n'
 ' Số chiều: [batch_size,8] \n'
 ' Kiểu dữ liệu: torch.FloatTensor \n'
 ' Các đặc trưng đầu vào:\n'
 " - 'MedInc' – Thu nhập trung vị\n"
 " - 'HouseAge' – Tuổi trung bình của căn nhà\n"
 " - 'AveRooms' – Số phòng trung bình\n"
 " - 'AveBedrms' – Số phòng ngủ trung bình\n"
 " - 'Population' – Dân số\n"
 " - 'AveOccup' – Số người trung bình trên mỗi hộ\n"
 " - 'Latitude' – Vĩ độ\n"
 " - 'Longitude' – Kinh độ\n"
 '\n'
 ' **📤 Output format (from tool):**  \n'
 ' 

In [473]:
pprint.pprint(ans.model_description)
pprint.pprint(ans.model_input_format)
pprint.pprint(ans.model_output_format)
pprint.pprint(ans.model_requirements)
pprint.pprint(ans.model_sample_code)

('Đây là một mô hình Wide & Deep Neural Network được huấn luyện trên tập dữ '
 'liệu California Housing để dự đoán giá nhà trung bình (MedHouseVal). Mô hình '
 'được xây dựng bằng PyTorch, dựa trên kiến trúc trong cuốn Hands-On Machine '
 'Learning with Scikit-Learn, Keras, and TensorFlow của Aurélien Géron.')
('Số chiều: [batch_size,8]\n'
 'Kiểu dữ liệu: torch.FloatTensor\n'
 'Các đặc trưng đầu vào:\n'
 "- 'MedInc' – Thu nhập trung vị\n"
 "- 'HouseAge' – Tuổi trung bình của căn nhà\n"
 "- 'AveRooms' – Số phòng trung bình\n"
 "- 'AveBedrms' – Số phòng ngủ trung bình\n"
 "- 'Population' – Dân số\n"
 "- 'AveOccup' – Số người trung bình trên mỗi hộ\n"
 "- 'Latitude' – Vĩ độ\n"
 "- 'Longitude' – Kinh độ")
('Kiểu: torch.FloatTensor có shape [batch_size,1]\n'
 'Ý nghĩa: Giá nhà trung bình dự đoán (giá trị thực).')
'None'
('import torch\n'
 'import torch.nn as nn\n'
 'from huggingface_hub import PyTorchModelHubMixin\n'
 '\n'
 '# Tạo dữ liệu đầu vào giả lập (batch1,8 features)\n'
 'x_input = t

In [474]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


In [475]:
class AgentCodingState(TypedDict):
    problem_description: str | None
    problem_input_description: str | None
    problem_output_description: str | None
    model_description: str | None
    model_input_format: str | None
    model_output_format: str | None
    model_requirements: str | None
    model_sample_code: str | None
    output_classes: str | None
    code: str | None


In [476]:
base_prompt = r"""
    You are a specialist in Machine Learning. Your task is to generate a *fully functional with all necessary imports and dependencies* codebase in *Python* that can be executed flawlessly.

    You will be provided with:
    - A problem description
    - An input specification
    - An output specification
    - A model description

    ### Input:
    - Problem description: {problem_description}
    - Problem input specification: {problem_input_description}
    - Problem output specification: {problem_output_description}
    - ML model description: {model_description}
    - ML model input format: {model_input_format}
    - ML model output format: {model_output_format}
    - ML model requirements: {model_requirements}
    - ML model sample code: {model_sample_code}
    - Output classes: {output_classes}

    ### Guidelines

    You *must* strictly follow the following guidelines:
    - The preprocessing step should be suitable for the data type.
    - The postprocessing step should notices the differences between the data returned by the model and the output requirements. You must extract and use the exact class labels as defined in the output specification.
    - Do *not invent new labels or translate* the class names. Use them exactly as given.
    - You *must* make sure that your codebase can be executed flawlessly that would not encounter any errors or exceptions.

    Your implementation *must strictly follow* the structure below:
    1. *Imports*: All required libraries.
    2. *Preprocessing*: Handle and transform the input as defined.
    3. *Inference logic*: Use the described model for prediction. You *must* use tqdm or similar logging library to track progress.
    4. *Postprocessing*: Format or transform the raw output into the final result as described.

    You must *not* include any explanations, markdown, or logging outside what is required by the problem.

    Return *only* the complete Python codebase, and you **MUST NOT** include a main function in any way. Wrap it with:
    \`\`\`python
    # code here
    \`\`\`
    """


In [477]:
class OutputState(TypedDict):
    output_classes: str | None

In [478]:
def output_agent(state: OutputState) -> OutputState:
    prompt = r"""
        Your are a specialist in machine learning. Your task is to identify the absolute classes of the given problem description, following with an output description.
        ### Input:
        - Problem description: {problem_description}
        - Output description: {output_description}

        You must return an array, strictly following these guidelines:
        - Understand the context from the given problem description.
        - Extract the class names from the output description. You **must not** invent new labels or translate the class names. Use them exactly as given in the output description.
        - Create an array containing the classes.

        You must return only the array containing those classes, without any formatting.
    """

    prompt = prompt.format(
        problem_description = state["problem_description"],
        output_description = state["output_description"]
    )
    response = llm.invoke(prompt)
    return {**state, "output_classes": response.content}

In [479]:
output_state = {
    "problem_description": task,
    "output_description": state["subtasks"].subtask_four
}

In [480]:
new_state = output_agent(output_state)

In [481]:
new_state["output_classes"]

'low, medium, high'

In [482]:
def coding_agent(state: AgentCodingState) -> AgentCodingState:
    prompt = base_prompt.format(
        problem_description=state["problem_description"],
        problem_input_description=state["problem_input_description"],
        problem_output_description=state["problem_output_description"],
        model_description=state["model_description"],
        model_input_format=state["model_input_format"],
        model_output_format=state["model_output_format"],
        model_requirements=state["model_requirements"],
        model_sample_code=state["model_sample_code"],
        output_classes=state["output_classes"],
    )
    response = llm.invoke(prompt)
    return {**state, "code": response.content}

In [483]:
prev_state = {
    "problem_description": task,
    "problem_input_description": state["subtasks"].subtask_three,
    "problem_output_description": state["subtasks"].subtask_four,
    "model_description": res2['output'],
    "model_input_format": ans.model_input_format,
    "model_output_format": ans.model_output_format,
    "model_requirements": ans.model_requirements,
    "model_sample_code": ans.model_sample_code,
    "output_classes": new_state["output_classes"]
}


In [484]:
coding_state = coding_agent(prev_state)

In [485]:
print(coding_state["code"])

```python
import pandas as pd
import torch
from huggingface_hub import PyTorchModelHubMixin
import torch.nn as nn

# Định nghĩa mô hình Wide & Deep Neural Network
class WideAndDeepNet(nn.Module, PyTorchModelHubMixin):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 30) # Sửa input size từ 6 thành 8
        self.hidden2 = nn.Linear(30, 30)
        self.main_head = nn.Linear(30, 1) # Sửa input size từ 35 thành 30
        self.aux_head = nn.Linear(30, 1)
        self.main_loss_fn = nn.MSELoss(reduction='sum')
        self.aux_loss_fn = nn.MSELoss(reduction='sum')

    def forward(self, input_deep, label=None):
        act = torch.relu(self.hidden1(input_deep))
        act = torch.relu(self.hidden2(act))
        main_output = self.main_head(act)
        aux_output = self.aux_head(act)
        if label is not None:
            main_loss = self.main_loss_fn(main_output.squeeze(), label)
            aux_loss = self.aux_loss_fn(aux_output.squeeze(), label

In [486]:
coding_state

{'problem_description': 'Bối cảnh của vấn đề:\nĐây là một nhiệm vụ hồi quy. Mục tiêu là dự đoán giá trị trung bình của nhà ở các quận của California bằng cách sử dụng các đặc trưng từ cuộc điều tra dân số năm 1990, chẳng hạn như thu nhập trung bình, tuổi trung bình của nhà ở, tổng số phòng, v.v. ()\n\nYêu cầu cụ thể cần đạt được:\nXây dựng một hệ thống có khả năng nhận đầu vào là một list các features của khu vực có liên quan đến giá trung bình của nhà ở và đầu ra hệ thống là phân loại khu vực đó thuộc nhóm “rẻ”, “trung bình” hay “đắt” để dễ trực quan hóa hoặc định hướng chính sách\nMô hình cần xử lý đầu vào là 1 list các features liên quan đến giá trung bình của nhà ở\nĐầu ra là nhãn của nhà ở khu vực đấy thuộc về\n\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\nMột file test.csv. Mỗi hàng là features cho từng task bao gồm các cột:\n\nMedInc: Thu nhập trung vị của người dân sống trong khu vực (block)\nHouseAge: Tuổi trung vị của các căn nhà trong khu vực\nAveRooms: Số phòng trung 